# Filter Itemid Prescriptions

First, we find the number of observation that has each unit of measurement. If the top frequent one has the observations less than 90% of total, then we discard it.

Also, there are many medicine that never be used by any patients, so we also discard it.

In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count

from utils import getConnection

%matplotlib inline

In [2]:
conn = getConnection()

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

In [3]:
def stat_prescript_unit_task(i, admission_ids_txt):
    conn = getConnection()
    # foreach medicine, list the dose unit of medicine and the count of observations that use that unit.
    cur = conn.cursor()
    cur.execute('SELECT dose_unit_rx, count(dose_unit_rx) FROM mimiciii.prescriptions WHERE formulary_drug_cd = \''+ str(i) +'\' and hadm_id in (select * from admission_ids) group by dose_unit_rx')
    outputunits = cur.fetchall()
    
    # sort it descendently
    outputunits = sorted(outputunits, key=lambda tup: tup[1])
    outputunits.reverse()
    return (i, outputunits)

#p = Pool(cpu_count() * 2)
#results = [p.apply_async(stat_prescript_unit_task, args=(i, admission_ids_txt)) for i in prescript_itemid]
results = [stat_prescript_unit_task(i, admission_ids_txt) for i in prescript_itemid]
#p.close()
#p.join()

#results = [x.get() for x in results]
np.save('res/filtered_prescript_raw.npy', {'raw': results})
print('saved!')

saved!


In [4]:
valid_prescript =[]
valid_prescript_unit =[]
dropped_id = []
notfound = []
results = np.load('res/filtered_prescript_raw.npy').tolist()['raw']

for x in results:
    i, outputunits = x[0], x[1]
    # check if medicine is never used by anybody, then discard it.
    total = 0
    for o in outputunits:
        total += o[1]
    if(total == 0 ): 
#         print("\n**PRES DROPPED "+str(i))
        notfound.append(i)
        for o in outputunits:
            print(o)
        print('')
        continue
        
    # calculate the percentage of observation of main unit.
    percentage = float(outputunits[0][1]) / total *100.
    if(percentage < 90): 
        # never drop the list A
#         if(i in manual_valid):
#             print("\n\n****PRES NOT DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
        
        # drop it !
#         else:
            dropped_id.append(i)
            print("\n\n****PRES DROPPED "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits)+"\n")
            continue
#     print("PRES "+str(i) + " : " + "{:.2f}".format(percentage) + " : " + str(len(outputunits))+" : "+ str(outputunits))
    
    # keep it and also save the main unit of it.
    valid_prescript.append(i)
    valid_prescript_unit.append(outputunits[0][0])



****PRES DROPPED GLYC50L : 88.89 : 2 : [('gm', 8), ('ml', 1)]



****PRES DROPPED BENZLIQ : 75.00 : 2 : [('Appl', 3), ('ml', 1)]



****PRES DROPPED ALDE22I : 54.05 : 2 : [('MU', 20), ('Million Units', 17)]



****PRES DROPPED CONJ625V : 84.21 : 2 : [('gm', 16), ('Appl', 3)]



****PRES DROPPED ALBU5500 : 80.47 : 3 : [('g', 3456), ('gm', 837), ('mL', 2)]



****PRES DROPPED NYST500T : 64.71 : 4 : [('UNIT', 110), ('TAB', 51), ('MU', 8), ('mL', 1)]



****PRES DROPPED METI.3O : 80.00 : 2 : [('DROP', 4), ('gtt', 1)]



****PRES DROPPED MYCOIND : 80.00 : 2 : [('CAP', 4), ('mg', 1)]



****PRES DROPPED DDAV1N : 64.85 : 3 : [('mcg', 107), ('SPRY', 57), ('PUFF', 1)]



****PRES DROPPED AMIN5L : 85.71 : 2 : [('gm', 6), ('ml', 1)]



****PRES DROPPED 0.45NS3000I : 68.38 : 2 : [('ml', 80), ('mL', 37)]



****PRES DROPPED SYRG1 : 75.53 : 6 : [('SYR', 3966), ('mL', 861), ('ml', 406), ('VIAL', 7), ('mg', 6), ('AMP', 5)]




****PRES DROPPED CIME150I : 75.00 : 2 : [('mg', 3), ('g', 1)]



****PRES

In [5]:
print(valid_prescript, valid_prescript_unit)
np.save('res/filtered_prescript.npy',{'id':valid_prescript,'unit':valid_prescript_unit})
print('saved!')

['NEOVIDAYLIN', 'PRED20', 'PROP100IG', 'DEXA0.5L', 'DOPA400D', 'IVIGBTL', 'METH2I', 'ACETSOLN', 'APRE125', 'SENOKOTS', 'MORP4I', 'D12.5W250I', 'SITA100', 'AA5D151000I', 'CHLO250', 'NEBU1000I', 'TRIM200R', 'BENGAY2O', 'PROP225', 'CAMP200I', 'RABI10I', 'ATOR80', 'CHLORO30', 'DARV100', 'FULV250IND', 'TEGR200', 'MYCO500I', 'APRA5OS', 'ESTR3', 'SCOP4I', 'DIG5P', 'MESA1000R', 'AMIT25', 'PROC100I', 'ADEN90I', 'SUNI50', 'F4B1ED', 'SILD25', 'METO10I', 'MAXISES', 'PERM5T', 'METH50L', 'TERAZ1', 'BENA10', 'MPRED5I', 'AGGRCAP', 'CLOM50', 'AMIN5', 'PRISMAK5000', 'FENT50I', 'FOLI1', 'CEFU250', 'ASCO250I', 'CALC100SI', 'FLUT110HFA', 'PERA200I', 'TROP5ES', 'MYCO200/1L', 'AUGM875L', 'CIDO1P', 'BISA5', 'ALTE20I', 'TAMS.4', 'ORBE5T', 'VANC.125L', 'AMLO25', 'NORT50', 'PHEN50', 'QUET50XR', 'CHLO500', 'CYPR4', 'EPHE50I', 'BUPI0.05EPD', 'RISP1L', 'NITR100', 'METH5', 'MESADESEN8', 'GCSF300I', 'MEPE50', 'ERGONEO', 'CHAR50L', 'NEOPOLYVI', 'MEPH32', 'METP250', 'DIGO200C', 'METL10', 'TOBR300H', 'ONDA4I', 'MORP1-10